In [2]:
!pip install vllm evaluate triton langchain_core langfuse -q -U
!pip install unsloth bert_score -q -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.5/255.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 

In [3]:
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-3dlq4r65
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-3dlq4r65
  Resolved https://github.com/unslothai/unsloth.git to commit bdf0cd6033595be4e7ed23d0d002bb176d343152
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.1.7-py3-none-any.whl size=174896 sha256=ea89a1420b0b15d6d39327542e76484d5cdd0687a61a85340d8d4c200cff589f
  Stored in directory: /tmp/pip-ephem-wheel-cache-d8hlkqp1/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.1.6
    Uninstalling unsloth-2025.1.6:
      Successfully uninstalled unsloth-2025.1.6


In [7]:
!pip install langdetect -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [8]:
import os
import json
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from langchain_core.prompts.prompt import PromptTemplate
from google.colab import drive, files
from langfuse import Langfuse
from unsloth import FastLanguageModel
from evaluate import load
from langdetect import detect, detect_langs

In [5]:
tqdm.pandas()

In [6]:
os.environ["LANGFUSE_SECRET_KEY"] = ""
os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

In [15]:
bertscore = load("bertscore")

In [ ]:
max_seq_length = 512
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-14B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.1.7: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/210k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

In [10]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
llm = LLM(model=model_name, max_model_len=10000, dtype="float16")

KeyboardInterrupt: 

In [12]:
langfuse_client = Langfuse()

In [13]:
DATASET_NAME = "aya_eval_ukr"

In [ ]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=512)

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
from datetime import datetime
from langfuse.decorators import langfuse_context, observe
from langfuse.client import DatasetItemClient

# @observe()
def _predict(example: DatasetItemClient):
    # langfuse_context.update_current_trace(
    #     input={
    #         "query": example.input["query"],
    #         "expected_output": example.expected_output["targets"],
    #     }
    # )
    input_str = example.input["query"]
    # outputs = llm.generate([input_str], sampling_params)
    formatted = "Дай відповідь на питання, враховуючи контекст, якщо він присутній.\nВідповідай лише українською.\n\n" + input_str

    inputs = tokenizer([formatted],return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    response_full = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response_full[len(formatted):]

    return response

    # print("\nOutputs\n")
    # print(outputs[0].outputs)
    # return outputs[0].outputs[0].text

def run_evaluation(
        dataset_name: str=DATASET_NAME,
    ):
        run_name="qwen-2.5-14b-int4-"+dataset_name+f"-{datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
        dataset = langfuse_client.get_dataset(dataset_name)
        for item in tqdm(dataset.items[:1], desc="Evaluation progress"):
          with item.observe(
                run_name=run_name
            ) as trace_id:
            reference=item.input["query"]
            print("\nInput:\n")
            print(reference)
            prediction = _predict(example=item)
            print("\nPrediction:\n")
            print(prediction)
            langfuse_client.trace(id=trace_id,
                                  input={
                "query": item.input["query"],
                "expected_output": item.expected_output["targets"],
            },
                                  output=prediction)
            bert_scores = bertscore.compute(predictions=[prediction], references=[reference], lang="uk")
            langfuse_client.score(trace_id=trace_id,
                                  name="bert_percision"
                                  value=bert_scores["precision"])

            langfuse_client.score(trace_id=trace_id,
                                  name="bert_recall"
                                  value=bert_scores["recall"])

            langfuse_client.score(trace_id=trace_id,
                                  name="bert_fq"
                                  value=bert_scores["f1"])
            print(results)

          # for name, metric_func in metrics.items():
          #   score = metric_func(response, item.expected_output)
          #   handler.trace.score(name=name, value=score)

In [ ]:
run_evaluation()

Evaluation progress:   0%|          | 0/1 [00:00<?, ?it/s]


Input:

Назвіть, будь ласка, основні світові джерела виробництва енергії.
Context:Виробництво енергії на 80% складається з викопних джерел, половину яких виробляють Китай, Сполучені Штати та арабські держави Перської затоки. Країни Перської затоки та Росія експортують більшу частину своєї продукції, переважно до Європейського Союзу та Китаю, де виробляється недостатньо енергії для задоволення попиту. Виробництво енергії зростає на 1-2% на рік, за винятком сонячної та вітрової енергії, які в середньому становили 20% на рік в 2010-х роках.

Prediction:

 Основні світові джерела виробництва енергії - це паливо (угольне, газове, нефтяне), атомна енергетика та регіональні джерела (наприклад, гідродинамічна енергетика). Зростання використання вітрової та сонячної енергії є трендом майбутнього. Основні світові джерела виробництва енергії - це угольне та газове паливо, нефть, атомна енергетика та гідродинамічна енергетика. Ці джерела загалом складають більшість виробництва енергії в світі. Со

Evaluation progress: 100%|██████████| 1/1 [01:57<00:00, 117.42s/it]
